In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from utils import Utils
#from utils import k_fold_cross
from smote import Smote
from logistic_regression_3 import LogisticRegression
from naive_bayes import NaiveBayes

In [2]:
## Reading the data
df = pd.read_csv('students_adaptability_level_online_education.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205 entries, 0 to 1204
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Gender               1205 non-null   object
 1   Age                  1205 non-null   object
 2   Education Level      1205 non-null   object
 3   Institution Type     1205 non-null   object
 4   IT Student           1205 non-null   object
 5   Location             1205 non-null   object
 6   Load-shedding        1205 non-null   object
 7   Financial Condition  1205 non-null   object
 8   Internet Type        1205 non-null   object
 9   Network Type         1205 non-null   object
 10  Class Duration       1205 non-null   object
 11  Self Lms             1205 non-null   object
 12  Device               1205 non-null   object
 13  Adaptivity Level     1205 non-null   object
dtypes: object(14)
memory usage: 131.9+ KB


In [3]:
# Checking the features
df.describe().T

,count,unique,top,freq
Gender,1205,2,Boy,663
Age,1205,6,21-25,374
Education Level,1205,3,School,530
Institution Type,1205,2,Non Government,823
IT Student,1205,2,No,901
Location,1205,2,Yes,935
Load-shedding,1205,2,Low,1004
Financial Condition,1205,3,Mid,878
Internet Type,1205,2,Mobile Data,695
Network Type,1205,3,4G,775


In [4]:
preprocesser = Utils(df)

In [5]:
## Preprocessing the categorical data and applying one-hot-encoding to the response
df.iloc[:, :-1] = preprocesser.labeling()
df = df.drop(['Adaptivity Level'], axis=1)
one_hot_response = preprocesser.one_hot_encoding()
#df = one_hot_response
df['Adaptivity Level_High'] = one_hot_response['Adaptivity Level_High']
df['Adaptivity Level_Low'] = one_hot_response['Adaptivity Level_Low']
df['Adaptivity Level_Moderate'] = one_hot_response['Adaptivity Level_Moderate']

/Users/besteguney/Documents/fourth_year/spring/eee485/Project_Github/EEE485/utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_df[column] = self.data[column].map(mapping)
/Users/besteguney/Documents/fourth_year/spring/eee485/Project_Github/EEE485/utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labeled_df[column] = self.data[column].map(mapping)
/Users/besteguney/Documents/fourth_year/spring/eee485/Project_Github/EEE485/utils.py:28: SettingWithCopyWarning: 
A value is trying to be 

In [6]:
## Handling imbalances in data
smote = Smote(df, 100, 5)
generated_data = smote.smote()
generated_data = pd.DataFrame(generated_data, columns=df.iloc[:, :-3].columns)
generated_data['Adaptivity Level_Low'] = 0
generated_data['Adaptivity Level_High'] = 1
generated_data['Adaptivity Level_Moderate'] = 0

<class 'numpy.ndarray'>


In [7]:
df = df.append(generated_data)

/var/folders/xn/ckzmp8n52bj64l343d38bl4r0000gn/T/ipykernel_20081/3962297237.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(generated_data)


In [8]:
df.reset_index(drop=True, inplace=True)

In [9]:
# Checking the features
df.tail()

,Gender,Age,Education Level,Institution Type,IT Student,Location,Load-shedding,Financial Condition,Internet Type,Network Type,Class Duration,Self Lms,Device,Adaptivity Level_High,Adaptivity Level_Low,Adaptivity Level_Moderate
1300,0,3,0,1,1,0,0,2,0,0,1,0,1,1,0,0
1301,0,3,0,1,1,0,0,2,0,0,1,0,2,1,0,0
1302,1,2,2,0,0,0,0,2,0,1,1,0,1,1,0,0
1303,0,0,0,0,1,0,1,1,0,0,1,1,2,1,0,0
1304,0,0,0,0,1,0,1,1,0,0,1,1,2,1,0,0


In [10]:
# Logistic Regression
logistic_regression = LogisticRegression(0.03, 10000)
print(logistic_regression.k_fold_cross(df, 10))

0.3923076923076923
0.23846153846153847
0.33076923076923076
0.36153846153846153
0.3230769230769231
0.2846153846153846
0.3076923076923077
0.2692307692307692
0.36153846153846153
0.3076923076923077
0.3176923076923077


In [11]:
# Naive Bayes
df['Adaptivity Level'] = df.iloc[:, -3:].idxmax(axis=1)
df['Adaptivity Level'] = df['Adaptivity Level'].map({'Adaptivity Level_High':2, 'Adaptivity Level_Low':0, 'Adaptivity Level_Moderate':1})
df = df.drop(['Adaptivity Level_High', 'Adaptivity Level_Moderate', 'Adaptivity Level_Low'], axis=1)

In [12]:
df

,Gender,Age,Education Level,Institution Type,IT Student,Location,Load-shedding,Financial Condition,Internet Type,Network Type,Class Duration,Self Lms,Device,Adaptivity Level
0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,1,0,1,0,1,1,1,1
2,1,1,1,1,0,0,0,0,0,0,1,0,1,1
3,1,2,2,0,0,0,0,0,1,0,1,0,1,1
4,1,1,2,0,0,0,0,1,1,1,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300,0,3,0,1,1,0,0,2,0,0,1,0,1,2
1301,0,3,0,1,1,0,0,2,0,0,1,0,2,2
1302,1,2,2,0,0,0,0,2,0,1,1,0,1,2
1303,0,0,0,0,1,0,1,1,0,0,1,1,2,2


In [13]:
nb = NaiveBayes()
nb.k_fold_cross(df, 10)

yvector (1305, 1)


0.35461538461538467